In [1]:
from datasets import load_dataset

dataset = load_dataset("roneneldan/TinyStories", split="train")

/opt/homebrew/Caskroom/miniforge/base/envs/jaxformer/lib/python3.11/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [2]:
from basic import tokenize, create_embeddings
import jax

rng = jax.random.PRNGKey(0)

train_data = dataset["text"][:200]
train_data, word2idx, vocab_size = tokenize(train_data)

max_len = 100

next_word_train_data = []
next_word_train_labels = []
for sentence in train_data:
    for i in range(1, min(len(sentence), max_len)):
        next_word_train_data.append(sentence[:i] + [0] * (max_len - i))
        next_word_train_labels.append(sentence[i])

In [3]:
import jax.numpy as jnp
from basic import onehot

next_word_train_data = jnp.array(next_word_train_data)
next_word_train_labels = jnp.array(next_word_train_labels)
next_word_train_labels = onehot(next_word_train_labels, vocab_size)

In [4]:
from basic import create_embeddings, get_model, init_model, num_params, batch_model

num_decoder_layers = 5
num_heads = 8
d_model = d_k = d_v = 64
seq_len = max_len

embeddings = create_embeddings(rng, vocab_size, d_model)
model = get_model(num_decoder_layers)
model = batch_model(model)
param_dict = init_model(rng, num_decoder_layers, num_heads, d_model, d_k, d_v, seq_len, vocab_size)
param_dict["embeddings"] = embeddings

print(num_params(param_dict))

26180649


In [ ]:
import jax
from basic import sgd_update, get_cross_entropy_loss

learning_rate = 0.005
num_epochs = 100
batch_size = 512
loss_fn = get_cross_entropy_loss(model)

for epoch in range(num_epochs):
    for i in range(0, len(next_word_train_data), batch_size):
        batch_data = next_word_train_data[i:i + batch_size]
        batch_labels = next_word_train_labels[i:i + batch_size]
        loss, grads = jax.value_and_grad(loss_fn)(param_dict, batch_data, batch_labels)
        param_dict = sgd_update(param_dict, grads, learning_rate)
        print(f"Batch {i // batch_size}: Loss {loss}")